# Import Library

In [2]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
# selenium의 webdriver를 사용하기 위한 import
from selenium import webdriver
from selenium.webdriver.common.by import By
# selenium으로 무엇인가 입력하기 위한 import
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
# 페이지 로딩을 기다리는데에 사용할 time 모듈 import
import time

크롤링 - 컨테이너의 사이즈와 무게

In [3]:
cwd = os.getcwd()
print('현재 폴더 경로 : ', cwd)
# 폴더 위치 변경
os.chdir("C:\\Users\hyunj\\Dropbox\\종합설계프로젝트\\중요\\데이터\\")
print('변경 후 폴더 경로 : ', cwd)
print('현재 폴더 파일 : ', os.listdir())
data = pd.read_csv('Container DS perspective.csv')
print('feature 항목 : ', data.columns)
data

현재 폴더 경로 :  c:\Users\hyunj\Desktop\종합설계프로젝트
변경 후 폴더 경로 :  c:\Users\hyunj\Desktop\종합설계프로젝트
현재 폴더 파일 :  ['Block perspective.csv', 'CHE I perspective.csv', 'CHE II perspective.csv', 'Container Data.csv', 'Container DS perspective.csv', 'Container LD perspective.csv', 'Container perspective.csv', 'Data.csv', 'DS.csv', 'DStoLD.csv', 'Dwell Time', 'Gate perspective.csv', 'GI perspective.csv', 'GO perspective.csv', 'ITV perspective.csv', 'LD.csv', 'QC perspective.csv', 'TC Data', 'TC Data.csv', 'TC224.csv', 'TC224_DS.csv', 'TC224_GI.csv', 'TC224_LD.csv', 'Vessel perspective.csv']
feature 항목 :  Index(['Unnamed: 0', 'VESSEL', 'CONTAINER_NO', 'MCHN_ID', 'MCHN_TP_CD',
       'FULL_EMPTY', 'JOB_TYPE', 'POD', 'BLOCK_BAY', 'JOB_START_DT',
       'JOB_END_DT'],
      dtype='object')


,Unnamed: 0,VESSEL,CONTAINER_NO,MCHN_ID,MCHN_TP_CD,FULL_EMPTY,JOB_TYPE,POD,BLOCK_BAY,JOB_START_DT,JOB_END_DT
0,1,ALAP-001/2018,AXIU1484082-2,GC112,QC,F,DS-QUAYSIDE,KRPUS,1B-38,20180131211752,20180131211924
1,2,ALAP-001/2018,AXIU1484082-2,TC221,TC,F,DS-YARDSIDE,KRPUS,1B-38,20180131213633,20180131213805
2,3,ALAP-001/2018,AXIU1484082-2,YT511,YT,F,DS-MOVE,KRPUS,1B-38,20180131211924,20180131213633
3,7,ALAP-001/2018,BGBU5060490-3,GC109,QC,F,DS-QUAYSIDE,KRPUS,3F-69,20180131205748,20180131205920
4,8,ALAP-001/2018,BGBU5060490-3,TC266,TC,F,DS-YARDSIDE,KRPUS,3F-69,20180131211132,20180131211304
...,...,...,...,...,...,...,...,...,...,...,...
223759,424301,SXNW-001/2018,UESU4153686-1,TC231,TC,F,DS-YARDSIDE,KRPUS,1C-10,20180129203032,20180129203204
223760,424302,SXNW-001/2018,UESU4153686-1,YT526,YT,F,DS-MOVE,KRPUS,1C-10,20180129201812,20180129203032
223761,424303,SXNW-001/2018,UESU4586161-1,GC110,QC,F,DS-QUAYSIDE,KRPUS,2B-74,20180129201945,20180129202117
223762,424304,SXNW-001/2018,UESU4586161-1,TC224,TC,F,DS-YARDSIDE,KRPUS,2B-74,20180129203317,20180129203449


In [4]:
# data['Container ID'] = data['CONTAINER_NO'][:-2]

for i in range(len(data)) :
    # data.loc[i, 'Container ID'] = data.loc[i, 'CONTAINER_NO'][:-2]
    temp = data.loc[i, 'VESSEL'].split('/')
    temp2 = temp[0].split('-')
    data.loc[i, 'VESSEL'] = temp2[0] + '-' + temp[1] + '-' + temp2[1]

In [5]:
# Container와 Vessel이 한 딕셔너리로 연결될 수 있게
container_vessel_dic = {}
for i in range(len(data)) : 
    container = data.loc[i, 'CONTAINER_NO']
    vessel = data.loc[i, 'VESSEL']
    container_vessel_dic[container] = vessel
    

# container_list = list(data['Container ID'].unique())
# container_list[5]

In [3]:
# 얼마나 데이터가 홈페이지와 매칭이 안되는지 판단하기 위한 변수
exception_number = 0
exception_number_list = []

driver = webdriver.Chrome('C:/chromedriver_win32/chromedriver.exe') 
driver.get('http://aris3.hjnc.co.kr:7110/hjnc_con')
time.sleep(1)


for container, vessel in tqdm(container_vessel_dic.items(), desc="크롤링 중~") :
    # 전체 동작은 컨테이너 번호 입력 -> 조회 버튼 클릭 -> 해당 VESSEL 클릭 -> 조회 버튼을 클릭하지 않아도 값이 바
    
    container_temp = container[:-2]
    container_search_box = driver.find_element(By.XPATH, '//*[@id="txtCntrNo"]')
    container_search_box.send_keys(container_temp)
    # container_search_box.send_keys(Keys.RETURN)
    
    # 조회 버튼 클릭
    driver.find_element(By.XPATH, '//*[@id="btnSearch"]').click()
    
    # Vessel 입력
    select_box = Select(driver.find_element(By.XPATH, '//*[@id="selVesselVoyage"]'))
    time.sleep(0.2) # 안 쉬어주면 오류 발생
    
    # 우리 데이터에는 있지만 홈페이지에는 없는 경우가 있음 이에 대해서는 패스해서 계속 크롤링을 하게 하였
    try :
        select_box.select_by_visible_text(vessel)
        # 검색어 초기화
        container_search_box.clear()
        # 값 추출
        size = driver.find_element(By.XPATH, '//*[@id="tblDetail"]/tbody/tr[4]/td[2]').text
        weight = driver.find_element(By.XPATH, '//*[@id="tblDetail"]/tbody/tr[5]/td[4]').text
        freeze = driver.find_element(By.XPATH, '//*[@id="tblDetail"]/tbody/tr[7]/td[2]').text
        imdg = driver.find_element(By.XPATH, '//*[@id="tblDetail"]/tbody/tr[6]/td[2]').text
        unno = driver.find_element(By.XPATH, '//*[@id="tblDetail"]/tbody/tr[6]/td[4]').text
        
        company = driver.find_element(By.XPATH, '//*[@id="tblDetail"]/tbody/tr[4]/td[4]').text
        import_or_export = driver.find_element(By.XPATH, '//*[@id="tblDetail"]/tbody/tr[3]/td[4]').text
        cargo_managemnet_no = driver.find_element(By.XPATH, '//*[@id="tblDetail"]/tbody/tr[12]/td[2]').text
        seal_no = driver.find_element(By.XPATH, '//*[@id="tblDetail"]/tbody/tr[9]/td[2]').text
        t_s = driver.find_element(By.XPATH, '//*[@id="tblDetail"]/tbody/tr[5]/td[2]').text
        
        # 값 입력
        #! 너무 많은 시간 복잡도
        # indexs = data.index[(data['Container ID'] == container)]
        # for index in indexs :
        index = data.index[(data['CONTAINER_NO'] == container)]
        data.loc[index, "SIZE"] = size
        data.loc[index, 'WEIGHT'] = weight
        data.loc[index, 'FREEZE'] = freeze
        data.loc[index, 'IMDG'] = imdg
        data.loc[index, 'UNNO'] = unno
        data.loc[index, "COMPANY"] = company
        data.loc[index, "IMPORT/EXPORT"] = import_or_export
        data.loc[index, "CARGO MANAGEMENT NUMBER"] = cargo_managemnet_no
        data.loc[index, "SEAL NUMBER"] = seal_no
        data.loc[index, "T/S"] = t_s
        
    except :
        exception_number += 1
        exception_number_list.append(container)
        container_search_box.clear()

NameError: name 'webdriver' is not defined

In [ ]:
data.to_csv('Container DS Data.csv', index=False, encoding='CP949')